# Scratchwork

In [28]:
from finite_algebras import make_finite_algebra, generate_prime_field
from finite_algebras import Group, Field

import numpy as np

In [2]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

## Checking for Field

### Rings

In [4]:
rng = make_finite_algebra('Powerset Ring 2',
                          'Ring on powerset of {0, 1}',
                          ['{}', '{0}', '{1}', '{0, 1}'],
                          [[0, 1, 2, 3],
                           [1, 0, 3, 2],
                           [2, 3, 0, 1],
                           [3, 2, 1, 0]],
                          [[0, 0, 0, 0],
                           [0, 1, 0, 1],
                           [0, 0, 2, 2],
                           [0, 1, 2, 3]]
                         )
rng.about()


Ring: Powerset Ring 2
Instance ID: 140589585375824
Description: Ring on powerset of {0, 1}
Identity: {}
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3  {0, 1}  {0, 1}       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: {0, 1}
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 2, 2], [0, 1, 2, 3]]


## Fields

$GF(4) = GF(2)[X]/(X^2 + X + 1)$

See Wikipedia: ["Field with four elements"](https://en.wikipedia.org/wiki/Finite_field#Field_with_four_elements)

In [31]:
elems = ['0', '1', 'a', '1+a']

add_table = [[ '0' ,  '1' ,  'a' , '1+a'],
             [ '1' ,  '0' , '1+a',  'a' ],
             [ 'a' , '1+a',  '0' ,  '1' ],
             ['1+a',  'a' ,  '1' ,  '0' ]]

mult_table = [['0',  '0' ,  '0' ,  '0' ],
              ['0',  '1' ,  'a' , '1+a'],
              ['0',  'a' , '1+a',  '1' ],
              ['0', '1+a',  '1' ,  'a' ]]

In [32]:
make_finite_algebra("foo", "FOOBAR", elems, add_table)

Group(
'foo',
'FOOBAR',
['0', '1', 'a', '1+a'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

In [33]:
f4 = Field('F4',
           'Field with 4 elements',
           elems,
           add_table,
           mult_table
          )

f4.about()


Field: F4
Instance ID: 140589051766032
Description: Field with 4 elements
Identity: 0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       1       1       2
      2       a       a       2
      3     1+a     1+a       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: 1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 3, 1], [0, 3, 1, 2]]


### Generate Field of Prime Order

In [11]:
f7 = generate_prime_field(7)
f7.about()


Ring: F7
Instance ID: 140589050915536
Description: Autogenerated prime field of order 7
Identity: a0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a0      a0       1
      1      a1      a6       7
      2      a2      a5       7
      3      a3      a4       7
      4      a4      a3       7
      5      a5      a2       7
      6      a6      a1       7
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5, 6, 0],
 [2, 3, 4, 5, 6, 0, 1],
 [3, 4, 5, 6, 0, 1, 2],
 [4, 5, 6, 0, 1, 2, 3],
 [5, 6, 0, 1, 2, 3, 4],
 [6, 0, 1, 2, 3, 4, 5]]
Mult. Identity: a1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 2, 4, 6, 1, 3, 5],
 [0, 3, 6, 2, 5, 1, 4],
 [0, 4, 1, 5, 2, 6, 3],
 [0, 5, 3, 1, 6, 4, 2],
 [0, 6, 5, 4, 3, 2, 1]]


In [12]:
f7_add = f7.extract_additive_algebra()
f7_add.about()


Group: F7.Add
Instance ID: 140589050878736
Description: Additive-only portion of F7
Identity: a0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a0      a0       1
      1      a1      a6       7
      2      a2      a5       7
      3      a3      a4       7
      4      a4      a3       7
      5      a5      a2       7
      6      a6      a1       7
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5, 6, 0],
 [2, 3, 4, 5, 6, 0, 1],
 [3, 4, 5, 6, 0, 1, 2],
 [4, 5, 6, 0, 1, 2, 3],
 [5, 6, 0, 1, 2, 3, 4],
 [6, 0, 1, 2, 3, 4, 5]]


In [13]:
f7_mult = f7.extract_multiplicative_algebra()
f7_mult.about()


Monoid: F7.Mult
Instance ID: 140589050877520
Description: Multiplicative-only portion of F7
Elements: ['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6']
Identity: a1
Associative? Yes
Commutative? Yes
Has Inverses? No
Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 2, 4, 6, 1, 3, 5],
 [0, 3, 6, 2, 5, 1, 4],
 [0, 4, 1, 5, 2, 6, 3],
 [0, 5, 3, 1, 6, 4, 2],
 [0, 6, 5, 4, 3, 2, 1]]


In [16]:
f7_mult_subs = f7_mult.proper_subalgebras()

print([(alg.__class__.__name__, alg.order) for alg in f7_mult_subs])

[alg for alg in f7_mult_subs if isinstance(alg, Group)]

[('Monoid', 4), ('Group', 6), ('Group', 3), ('Monoid', 2), ('Monoid', 3), ('Group', 2)]


[Group(
 'F7.Mult_subalgebra_1',
 'Subalgebra of: Multiplicative-only portion of F7',
 ['a1', 'a2', 'a3', 'a4', 'a5', 'a6'],
 [[0, 1, 2, 3, 4, 5], [1, 3, 5, 0, 2, 4], [2, 5, 1, 4, 0, 3], [3, 0, 4, 1, 5, 2], [4, 2, 0, 5, 3, 1], [5, 4, 3, 2, 1, 0]]
 ),
 Group(
 'F7.Mult_subalgebra_2',
 'Subalgebra of: Multiplicative-only portion of F7',
 ['a1', 'a2', 'a4'],
 [[0, 1, 2], [1, 2, 0], [2, 0, 1]]
 ),
 Group(
 'F7.Mult_subalgebra_5',
 'Subalgebra of: Multiplicative-only portion of F7',
 ['a1', 'a6'],
 [[0, 1], [1, 0]]
 )]

In [17]:
f7_mult_sub2 = f7_mult_subs[2]
f7_mult_sub2.about()


Group: F7.Mult_subalgebra_2
Instance ID: 140589049111376
Description: Subalgebra of: Multiplicative-only portion of F7
Identity: a1
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a1      a1       1
      1      a2      a4       3
      2      a4      a2       3
Cayley Table (showing indices):
[[0, 1, 2], [1, 2, 0], [2, 0, 1]]


In [18]:
rng = f7
rng.about()


Ring: F7
Instance ID: 140589050915536
Description: Autogenerated prime field of order 7
Identity: a0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a0      a0       1
      1      a1      a6       7
      2      a2      a5       7
      3      a3      a4       7
      4      a4      a3       7
      5      a5      a2       7
      6      a6      a1       7
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5, 6, 0],
 [2, 3, 4, 5, 6, 0, 1],
 [3, 4, 5, 6, 0, 1, 2],
 [4, 5, 6, 0, 1, 2, 3],
 [5, 6, 0, 1, 2, 3, 4],
 [6, 0, 1, 2, 3, 4, 5]]
Mult. Identity: a1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 2, 4, 6, 1, 3, 5],
 [0, 3, 6, 2, 5, 1, 4],
 [0, 4, 1, 5, 2, 6, 3],
 [0, 5, 3, 1, 6, 4, 2],
 [0, 6, 5, 4, 3, 2, 1]]


In [19]:
rng_mult = f7_mult
rng_mult.about()


Monoid: F7.Mult
Instance ID: 140589050877520
Description: Multiplicative-only portion of F7
Elements: ['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6']
Identity: a1
Associative? Yes
Commutative? Yes
Has Inverses? No
Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 2, 4, 6, 1, 3, 5],
 [0, 3, 6, 2, 5, 1, 4],
 [0, 4, 1, 5, 2, 6, 3],
 [0, 5, 3, 1, 6, 4, 2],
 [0, 6, 5, 4, 3, 2, 1]]


In [20]:
add_id = rng.add_identity
add_id

'a0'

In [21]:
# rng_mult = make_finite_algebra_2("tmp", "Temp", elements, mult_table.table)

In [22]:
elems = rng_mult.elements.copy()
elems

['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6']

In [23]:
elems.remove(add_id)
elems

['a1', 'a2', 'a3', 'a4', 'a5', 'a6']

In [24]:
elems_closed = sorted(rng_mult.closure(elems))
elems_closed

['a1', 'a2', 'a3', 'a4', 'a5', 'a6']

In [25]:
set(elems) == set(elems_closed)

True

In [26]:
alg = rng_mult.subalgebra_from_elements(elems)
alg

Group(
'No name',
'No description',
['a1', 'a2', 'a3', 'a4', 'a5', 'a6'],
[[0, 1, 2, 3, 4, 5], [1, 3, 5, 0, 2, 4], [2, 5, 1, 4, 0, 3], [3, 0, 4, 1, 5, 2], [4, 2, 0, 5, 3, 1], [5, 4, 3, 2, 1, 0]]
)

In [27]:
isinstance(alg, Group) and alg.is_commutative()

True